In [12]:
# =============================================================================
# Project: Arabic Sentiment Analysis (Mini-Scale)
# Description: A rule-based NLP project using Naive Bayes to classify Arabic 
#              product reviews. Features custom text normalization and 
#              N-gram vectorization to handle small-data context.
# Maria Alsadiq
# =============================================================================

import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------------------------------------------------------
# 1. Dataset Construction (Balanced Approach)
# -----------------------------------------------------------------------------
# Creating a balanced dataset manually to prevent class bias (Positive vs Negative)
csv_content = """text,sentiment
المنتج رائع جداً وأنصح به,Positive
سعيد جداً بهذه التجربة المميزة,Positive
شكراً لكم على سرعة التوصيل,Positive
جودة ممتازة وسعر مناسب,Positive
خدمة العملاء ممتازة وسريعة,Positive
وصل المنتج بحالة سليمة وجيدة,Positive
الجهاز يعمل بشكل ممتاز,Positive
تجربة جميلة وممتعة,Positive
منتج جيد ويستحق الشراء,Positive
كل شيء كان مثالياً,Positive
أفضل متجر تعاملت معه,Positive
المنتج وصل بسرعة وهو جيد جدا,Positive
أحببت هذا المنتج كثيرا,Positive
الخامة ممتازة جدا,Positive
توصيل سريع وتعامل رائع,Positive
منتج أصلي ورائع,Positive
سأطلب مرة أخرى بالتأكيد,Positive
تجربة شرائية ناجحة,Positive
خدمة رائعة وسريعة,Positive
سعر رخيص وجودة عالية,Positive
المنتج جميل جداً وفخم,Positive
شكرا على التعامل الراقي,Positive
أنصح الجميع بالتعامل معكم,Positive
منتج يستحق كل ريال,Positive
التغليف ممتاز والمنتج سليم,Positive
خدمة العملاء قمة في الاخلاق,Positive
تجربة جميلة و المنتج جيد,Positive
خدمة العملاء جيده جدا,Positive
المنتج سيء ولا يعمل,Negative
خسارة فلوس على الفاضي,Negative
خدمة سيئة جداً وتأخير,Negative
المنتج وصل مكسور,Negative
لا أنصح بالشراء أبداً,Negative
الجودة رديئة جداً,Negative
تجربة مروعة ولن أكررها,Negative
أسوأ تجربة في حياتي,Negative
تغليف سيء والمنتج متضرر,Negative
بطيء جداً ويعلق,Negative
خدمة العملاء لا يردون,Negative
المنتج غير مطابق للصورة,Negative
تجربة جميلة لكن المنتج سيء,Negative
المنتج وصل تالف,Negative
ندمت على الشراء,Negative
جودة سيئة جدا لا انصح به,Negative
المنتج مقلد وليس أصلي,Negative
تجربة فاشلة بكل المقاييس,Negative
كانت التجربة كارثية,Negative
تأخير في الشحنة وتغليف سيء,Negative
لم يعجبني المنتج نهائيا,Negative
لون المنتج باهت وغير جيد,Negative
خامته رديئة جدا وسعره غالي,Negative
المنتج وصل مكسور الكرتون,Negative
سيء جدا جدا,Negative
لا يستحق سعره,Negative
خدمة ما بعد البيع سيئة,Negative
"""

# Saving dataset locally
with open('arabic_reviews.csv', 'w', encoding='utf-8') as f:
    f.write(csv_content)

print("Dataset created successfully.")

# -----------------------------------------------------------------------------
# 2. Data Preprocessing (Arabic Normalization)
# -----------------------------------------------------------------------------
df = pd.read_csv('arabic_reviews.csv')

def clean_text(text):
    """
    Normalizes Arabic text by unifying different forms of letters.
    -  (أ، إ، آ) -> ا
    -  (ة) -> ه
    -  (ى) -> ي
    """
    text = str(text)
    text = re.sub(r'[أإآ]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

# -----------------------------------------------------------------------------
# 3. Model Pipeline & Training
# -----------------------------------------------------------------------------
# Using N-grams (1,2) to capture context like "not good" or "very bad"
model = make_pipeline(CountVectorizer(ngram_range=(1, 2)), MultinomialNB())

# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['sentiment'], test_size=0.1, random_state=42)

# Training
model.fit(X_train, y_train)
print("Model trained successfully.")

# -----------------------------------------------------------------------------
# 4. Evaluation & Live Testing
# -----------------------------------------------------------------------------
print("\n--- Live Inference ---")
test_cases = [
    "المنتج وصل انه رائع",           # Tricky case 1
    "المنتج وصل وهو سيء جدا",      # Tricky case 2
    "خدمة العملاء جيده",          # Normalization test
    "كانت التجربة كارثية",        # Vocabulary test
    "تجربة جميلة و المنتج جيد"    # Compound sentence
]

for text in test_cases:
    cleaned = clean_text(text)
    prediction = model.predict([cleaned])[0]
    prob = model.predict_proba([cleaned]).max() * 100
    
    print(f"Text: {text}")
    print(f"Prediction: {prediction} (Confidence: {prob:.1f}%)")
    print("-" * 30)

Dataset created successfully.
Model trained successfully.

--- Live Inference ---
Text: المنتج وصل انه رائع
Prediction: Negative (Confidence: 52.0%)
------------------------------
Text: المنتج وصل وهو سيء جدا
Prediction: Negative (Confidence: 96.6%)
------------------------------
Text: خدمة العملاء جيده
Prediction: Positive (Confidence: 94.7%)
------------------------------
Text: كانت التجربة كارثية
Prediction: Negative (Confidence: 94.7%)
------------------------------
Text: تجربة جميلة و المنتج جيد
Prediction: Positive (Confidence: 94.8%)
------------------------------


In [13]:
# ==========================================
# 5. Interactive Mode (Try it yourself!) 
# ==========================================
print("\n--- نظام تحليل المشاعر العربي (تجربة تفاعلية) ---")
print("اكتب جملتك للتحليل، أو اكتب 'خروج' للإنهاء.\n")

while True:
    user_input = input("أدخل النص هنا: ")
   
    if user_input.lower() in ['خروج', 'exit', 'quit']:
        print(" شكرا لك ")
        break
        
    if not user_input.strip():
        continue

    cleaned_input = clean_text(user_input)
    
    prediction = model.predict([cleaned_input])[0]
    
    prob = model.predict_proba([cleaned_input]).max() * 100

    emoji = " 😁 " if prediction == "Positive" or prediction == "إيجابي" else "😡"
    print(f"  النص: {user_input}")
    print(f"  التحليل: {prediction} {emoji}")
    print(f"  الثقة: {prob:.1f}%")
    print("-" * 30)


--- نظام تحليل المشاعر العربي (تجربة تفاعلية) ---
اكتب جملتك للتحليل، أو اكتب 'خروج' للإنهاء.



أدخل النص هنا:  التجربة كانت سيئة


  النص: التجربة كانت سيئة
  التحليل: Negative 😡
  الثقة: 89.4%
------------------------------


أدخل النص هنا:  هذا رائع !!


  النص: هذا رائع !!
  التحليل: Positive  😁 
  الثقة: 85.5%
------------------------------


...

  النص: خدمة العملاء جيدة
  التحليل: Positive  😁 
  الثقة: 94.7%
------------------------------


أدخل النص هنا:  الردود كانت سيئة


  النص: الردود كانت سيئة
  التحليل: Negative 😡
  الثقة: 89.1%
------------------------------


أدخل النص هنا:  خروج


 شكرا لك 
